In [2]:
import numpy as np

def create_matrix(n):
    A = np.zeros((n, n))
    for i in range(n):
        A[i, i] = 1  # Main diagonal elements = 1
        if i < n - 1:
            A[i, i + 1] = 2  # First superdiagonal elements = 2
    return A

def norm_1(A): # (Maximum of column sums)
    return np.max(np.sum(np.abs(A), axis=0))

def norm_inf(A): # (Maximum of row sums)
    return np.max(np.sum(np.abs(A), axis=1))

def initial_matrix_V0(A): # V0 = A^T / (||A||₁ * ||A||_∞)
    return A.T / (norm_1(A) * norm_inf(A))

# Verification prints
n = 4  # Example size for matrix A
print("=" * 60)
print("CREATING THE SPECIAL MATRIX A")
print("=" * 60)
print("This matrix has 1's on the main diagonal and 2's on the first superdiagonal.")
print("It is the matrix we need to find the inverse for.")
A = create_matrix(n)
print("\nMatrix A (4x4):")
print(A)

print("\n" + "=" * 60)
print("CALCULATING MATRIX NORMS (needed for initial approximation)")
print("=" * 60)
print("Norm-1 (maximum column sum):", norm_1(A))
print("Norm-infinity (maximum row sum):", norm_inf(A))

print("\n" + "=" * 60)
print("INITIAL APPROXIMATION OF THE INVERSE")
print("=" * 60)
print("V0 = A^T / (||A||₁ * ||A||_∞) [Formula (5) from the problem]")
print("This will be our starting matrix for the iterative methods.")
V0 = initial_matrix_V0(A)
print("\nInitial matrix V0:")
print(V0)

CREATING THE SPECIAL MATRIX A
This matrix has 1's on the main diagonal and 2's on the first superdiagonal.
It is the matrix we need to find the inverse for.

Matrix A (4x4):
[[1. 2. 0. 0.]
 [0. 1. 2. 0.]
 [0. 0. 1. 2.]
 [0. 0. 0. 1.]]

CALCULATING MATRIX NORMS (needed for initial approximation)
Norm-1 (maximum column sum): 3.0
Norm-infinity (maximum row sum): 3.0

INITIAL APPROXIMATION OF THE INVERSE
V0 = A^T / (||A||₁ * ||A||_∞) [Formula (5) from the problem]
This will be our starting matrix for the iterative methods.

Initial matrix V0:
[[0.11111111 0.         0.         0.        ]
 [0.22222222 0.11111111 0.         0.        ]
 [0.         0.22222222 0.11111111 0.        ]
 [0.         0.         0.22222222 0.11111111]]


In [3]:
def schultz_method(A, V0, epsilon, kmax):
    """
    Schultz method (also known as Hotelling-Bodewig)
    Formula (1): V_{k+1} = V_k(2I_n - AV_k)

    This is an iterative method to approximate the inverse of matrix A.
    """
    n = A.shape[0]
    I = np.eye(n)  # Identity matrix of size n
    V_prev = V0.copy()  # Start with initial approximation
    k = 0  # Iteration counter

    while k < kmax:
        # Calculate V_{k+1} = V_k(2I_n - AV_k)
        AV = A @ V_prev  # Matrix multiplication A*V_k
        V_next = V_prev @ (2 * I - AV)  # New approximation V_{k+1}

        # Calculate difference norm to check convergence
        diff_norm = np.linalg.norm(V_next - V_prev, np.inf)

        # Check convergence criteria
        if diff_norm < epsilon:
            return V_next, k + 1, True  # Converged successfully
        elif diff_norm > 10**10:
            return V_next, k + 1, False  # Diverged (growing too large)

        V_prev = V_next.copy()  # Update for next iteration
        k += 1

    return V_prev, k, False  # Max iterations reached without convergence

def li_li_method1(A, V0, epsilon, kmax):
    """
    Li and Li method 1
    Formula (2): V_{k+1} = V_k(3I_n - AV_k(3I_n - AV_k))

    An alternative iterative method that may converge faster in some cases.
    """
    n = A.shape[0]
    I = np.eye(n)  # Identity matrix of size n
    V_prev = V0.copy()  # Start with initial approximation
    k = 0  # Iteration counter

    while k < kmax:
        # Calculate V_{k+1} = V_k(3I_n - AV_k(3I_n - AV_k))
        AV = A @ V_prev  # Matrix multiplication A*V_k
        inner_term = 3 * I - AV  # Calculate (3I_n - AV_k) first
        V_next = V_prev @ (3 * I - AV @ inner_term)  # New approximation V_{k+1}

        # Calculate difference norm to check convergence
        diff_norm = np.linalg.norm(V_next - V_prev, np.inf)

        # Check convergence criteria
        if diff_norm < epsilon:
            return V_next, k + 1, True  # Converged successfully
        elif diff_norm > 10**10:
            return V_next, k + 1, False  # Diverged (growing too large)

        V_prev = V_next.copy()  # Update for next iteration
        k += 1

    return V_prev, k, False  # Max iterations reached without convergence

# Test with the example matrix
n = 4
print("\n" + "=" * 60)
print("TESTING ITERATIVE METHODS FOR MATRIX INVERSION")
print("=" * 60)

A = create_matrix(n)
print(f"Using our special {n}x{n} matrix A:")
print(A)

V0 = initial_matrix_V0(A)
print("\nStarting with initial approximation V0:")
print(V0)

epsilon = 1e-8
print(f"\nConvergence criteria: ||V_k+1 - V_k|| < {epsilon}")

kmax = 100
print(f"Maximum iterations allowed: {kmax}")

# Test Schultz method
print("\n" + "=" * 60)
print("METHOD 1: SCHULTZ METHOD (HOTELLING-BODEWIG)")
print("=" * 60)
print("Formula: V_{k+1} = V_k(2I_n - AV_k)")
V_schultz, k_schultz, converged_schultz = schultz_method(A, V0, epsilon, kmax)

if converged_schultz:
    print(f"\nSUCCESS! Method converged after {k_schultz} iterations.")
else:
    if k_schultz >= kmax:
        print(f"\nWARNING: Method did not converge after maximum {kmax} iterations.")
    else:
        print(f"\nWARNING: Method diverged after {k_schultz} iterations.")

print("\nApproximated inverse matrix:")
print(V_schultz)

# Calculate error norm ||A*A^(-1) - I||
error_norm = np.linalg.norm(A @ V_schultz - np.eye(n), 1)
print(f"\nError measure ||A*A^(-1) - I||_1 = {error_norm:.8f}")
if error_norm < epsilon:
    print("This is a good approximation of the inverse!")
else:
    print("The approximation could be better. Consider using more iterations or another method.")

# Test Li and Li method 1
print("\n" + "=" * 60)
print("METHOD 2: LI AND LI METHOD 1")
print("=" * 60)
print("Formula: V_{k+1} = V_k(3I_n - AV_k(3I_n - AV_k))")
V_li1, k_li1, converged_li1 = li_li_method1(A, V0, epsilon, kmax)

if converged_li1:
    print(f"\nSUCCESS! Method converged after {k_li1} iterations.")
else:
    if k_li1 >= kmax:
        print(f"\nWARNING: Method did not converge after maximum {kmax} iterations.")
    else:
        print(f"\nWARNING: Method diverged after {k_li1} iterations.")

print("\nApproximated inverse matrix:")
print(V_li1)

# Calculate error norm ||A*A^(-1) - I||
error_norm = np.linalg.norm(A @ V_li1 - np.eye(n), 1)
print(f"\nError measure ||A*A^(-1) - I||_1 = {error_norm:.8f}")
if error_norm < epsilon:
    print("This is a good approximation of the inverse!")
else:
    print("The approximation could be better. Consider using more iterations or another method.")

# Compare the two methods
print("\n" + "=" * 60)
print("COMPARISON OF METHODS")
print("=" * 60)
print(f"Schultz method required {k_schultz} iterations")
print(f"Li and Li method 1 required {k_li1} iterations")

if k_schultz < k_li1:
    print("For this matrix, Schultz method converged faster!")
elif k_li1 < k_schultz:
    print("For this matrix, Li and Li method 1 converged faster!")
else:
    print("Both methods required the same number of iterations.")


TESTING ITERATIVE METHODS FOR MATRIX INVERSION
Using our special 4x4 matrix A:
[[1. 2. 0. 0.]
 [0. 1. 2. 0.]
 [0. 0. 1. 2.]
 [0. 0. 0. 1.]]

Starting with initial approximation V0:
[[0.11111111 0.         0.         0.        ]
 [0.22222222 0.11111111 0.         0.        ]
 [0.         0.22222222 0.11111111 0.        ]
 [0.         0.         0.22222222 0.11111111]]

Convergence criteria: ||V_k+1 - V_k|| < 1e-08
Maximum iterations allowed: 100

METHOD 1: SCHULTZ METHOD (HOTELLING-BODEWIG)
Formula: V_{k+1} = V_k(2I_n - AV_k)

SUCCESS! Method converged after 16 iterations.

Approximated inverse matrix:
[[ 1.00000000e+00 -2.00000000e+00  4.00000000e+00 -8.00000000e+00]
 [ 1.45434799e-29  1.00000000e+00 -2.00000000e+00  4.00000000e+00]
 [-6.68144552e-30  1.77156230e-29  1.00000000e+00 -2.00000000e+00]
 [ 2.70202537e-30 -7.04140569e-30  1.41875768e-29  1.00000000e+00]]

Error measure ||A*A^(-1) - I||_1 = 0.00000000
This is a good approximation of the inverse!

METHOD 2: LI AND LI METHOD 1

In [4]:
def li_li_method2(A, V0, epsilon, kmax):
    """
    Li and Li method 2
    Formula (3): V_{k+1} = (I_n + (1/4)(I_n - V_k*A)(3I_n - V_k*A)^2)*V_k

    This is another iterative method that may provide faster convergence
    for certain types of matrices.
    """
    n = A.shape[0]
    I = np.eye(n)  # Identity matrix of size n
    V_prev = V0.copy()  # Start with initial approximation
    k = 0  # Iteration counter

    while k < kmax:
        # Calculate V_{k+1} using formula (3)
        VA = V_prev @ A  # Matrix multiplication V_k*A
        I_minus_VA = I - VA  # Calculate (I_n - V_k*A)
        term1 = 3*I - VA  # Calculate (3I_n - V_k*A)
        term2 = term1 @ term1  # Square the term (3I_n - V_k*A)^2
        term3 = I_minus_VA @ term2  # Multiply (I_n - V_k*A)(3I_n - V_k*A)^2
        V_next = (I + 0.25 * term3) @ V_prev  # Calculate final V_{k+1}

        # Calculate difference norm to check convergence
        diff_norm = np.linalg.norm(V_next - V_prev, np.inf)

        # Check convergence criteria
        if diff_norm < epsilon:
            return V_next, k + 1, True  # Converged successfully
        elif diff_norm > 10**10:
            return V_next, k + 1, False  # Diverged (growing too large)

        V_prev = V_next.copy()  # Update for next iteration
        k += 1

    return V_prev, k, False  # Max iterations reached without convergence

def calculate_error(A, V_approx):
    """
    Calculate the error norm: ||A*A_approx^(-1) - I||_1

    This measures how close our approximated inverse is to the true inverse.
    If A*A^(-1) = I (identity matrix), then the error would be 0.
    The smaller this value, the better our approximation.
    """
    n = A.shape[0]
    I = np.eye(n)  # Identity matrix
    error = np.linalg.norm(A @ V_approx - I, 1)  # Calculate error using 1-norm
    return error

# Test Li and Li method 2
print("\n" + "=" * 60)
print("METHOD 3: LI AND LI METHOD 2")
print("=" * 60)
print("Formula: V_{k+1} = (I_n + (1/4)(I_n - V_k*A)(3I_n - V_k*A)^2)*V_k")
print("This is often considered the most sophisticated of the three methods.")

V_li2, k_li2, converged_li2 = li_li_method2(A, V0, epsilon, kmax)

if converged_li2:
    print(f"\nSUCCESS! Method converged after {k_li2} iterations.")
else:
    if k_li2 >= kmax:
        print(f"\nWARNING: Method did not converge after maximum {kmax} iterations.")
    else:
        print(f"\nWARNING: Method diverged after {k_li2} iterations.")

print("\nApproximated inverse matrix:")
print(V_li2)

# Calculate error norm for Li and Li method 2
error_li2 = calculate_error(A, V_li2)
print(f"\nError measure ||A*A^(-1) - I||_1 = {error_li2:.8f}")
if error_li2 < epsilon:
    print("This is a good approximation of the inverse!")
else:
    print("The approximation could be better. Consider using more iterations.")

# Calculate errors for all methods
print("\n" + "=" * 60)
print("COMPREHENSIVE COMPARISON OF ALL THREE METHODS")
print("=" * 60)

error_schultz = calculate_error(A, V_schultz)
error_li1 = calculate_error(A, V_li1)
error_li2 = calculate_error(A, V_li2)

print("Error norms (lower is better):")
print(f"1. Schultz method:     {error_schultz:.8f} ({k_schultz} iterations)")
print(f"2. Li and Li method 1: {error_li1:.8f} ({k_li1} iterations)")
print(f"3. Li and Li method 2: {error_li2:.8f} ({k_li2} iterations)")

# Determine the best method
best_error = min(error_schultz, error_li1, error_li2)
if best_error == error_schultz:
    best_method = "Schultz method"
    best_iter = k_schultz
elif best_error == error_li1:
    best_method = "Li and Li method 1"
    best_iter = k_li1
else:
    best_method = "Li and Li method 2"
    best_iter = k_li2

print(f"\nThe best approximation was provided by: {best_method}")
print(f"With error: {best_error:.8f} after {best_iter} iterations")

# Compare convergence speed
fastest_method = min([(k_schultz, "Schultz method"),
                      (k_li1, "Li and Li method 1"),
                      (k_li2, "Li and Li method 2")])

print(f"\nThe fastest convergence was achieved by: {fastest_method[1]}")
print(f"With only {fastest_method[0]} iterations required")


METHOD 3: LI AND LI METHOD 2
Formula: V_{k+1} = (I_n + (1/4)(I_n - V_k*A)(3I_n - V_k*A)^2)*V_k
This is often considered the most sophisticated of the three methods.

SUCCESS! Method converged after 10 iterations.

Approximated inverse matrix:
[[ 1.00000000e+00 -2.00000000e+00  4.00000000e+00 -8.00000000e+00]
 [-4.08541542e-32  1.00000000e+00 -2.00000000e+00  4.00000000e+00]
 [ 1.37143848e-32 -1.08401594e-31  1.00000000e+00 -2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Error measure ||A*A^(-1) - I||_1 = 0.00000000
This is a good approximation of the inverse!

COMPREHENSIVE COMPARISON OF ALL THREE METHODS
Error norms (lower is better):
1. Schultz method:     0.00000000 (16 iterations)
2. Li and Li method 1: 0.00000000 (11 iterations)
3. Li and Li method 2: 0.00000000 (10 iterations)

The best approximation was provided by: Li and Li method 1
With error: 0.00000000 after 11 iterations

The fastest convergence was achieved by: Li and Li method 2
Wit

In [5]:
def exact_inverse(n):
    # Through experimentation, we've determined that the inverse has:
    # - 1's on the main diagonal
    # - For elements above the diagonal: (-2)^(j-i) where j > i

    inv = np.zeros((n, n))

    # Fill the main diagonal with 1's
    for i in range(n):
        inv[i, i] = 1

    # Fill the upper triangular part according to the pattern (-2)^k. Derived from experiments
    for i in range(n):
        for j in range(i+1, n):
            # The element at position (i,j) equals (-2) raised to the power (j-i)
            inv[i, j] = (-2)**(j-i)

    return inv

def compare_methods_with_different_n(n_values, epsilon, kmax):
    results = []

    for n in n_values:
        print(f"\n{'-'*30}")
        print(f"ANALYZING MATRIX SIZE n = {n}")
        print(f"{'-'*30}")

        # Create the special matrix A and compute initial approximation
        A = create_matrix(n)
        V0 = initial_matrix_V0(A)

        # Calculate the exact inverse using our formula
        A_inv_exact = exact_inverse(n)
        print(f"Created {n}x{n} matrix A and calculated its exact inverse")

        # Run all three methods
        print("\nRunning the three iterative methods...")
        V_schultz, k_schultz, conv_schultz = schultz_method(A, V0, epsilon, kmax)
        V_li1, k_li1, conv_li1 = li_li_method1(A, V0, epsilon, kmax)
        V_li2, k_li2, conv_li2 = li_li_method2(A, V0, epsilon, kmax)

        # Calculate errors compared to exact inverse
        error_exact_schultz = np.linalg.norm(V_schultz - A_inv_exact, np.inf)
        error_exact_li1 = np.linalg.norm(V_li1 - A_inv_exact, np.inf)
        error_exact_li2 = np.linalg.norm(V_li2 - A_inv_exact, np.inf)

        print(f"Schultz method: {k_schultz} iterations, Error vs. exact: {error_exact_schultz:.8f}")
        print(f"Li & Li method 1: {k_li1} iterations, Error vs. exact: {error_exact_li1:.8f}")
        print(f"Li & Li method 2: {k_li2} iterations, Error vs. exact: {error_exact_li2:.8f}")

        # Store results for later comparison
        results.append({
            'n': n,
            'iterations_schultz': k_schultz,
            'iterations_li1': k_li1,
            'iterations_li2': k_li2,
            'converged_schultz': conv_schultz,
            'converged_li1': conv_li1,
            'converged_li2': conv_li2,
            'error_exact_schultz': error_exact_schultz,
            'error_exact_li1': error_exact_li1,
            'error_exact_li2': error_exact_li2
        })

    return results

# Test the exact inverse calculation
print("\n" + "=" * 60)
print("DERIVING AND VERIFYING THE EXACT INVERSE FORMULA")
print("=" * 60)
print("For the special matrix A, we've analyzed its pattern and determined")
print("an exact formula for its inverse without using iterative methods.")

n = 4
A = create_matrix(n)
print("\nOriginal matrix A:")
print(A)

A_inv_exact = exact_inverse(n)
print("\nExact inverse matrix derived using our formula:")
print(A_inv_exact)

# Verify by multiplying A and A^(-1)
product = A @ A_inv_exact
print("\nVerification: A * A^(-1) should equal the identity matrix:")
print(product)

# Calculate the error in the verification
error_verification = np.linalg.norm(product - np.eye(n), np.inf)
print(f"\nError in verification: {error_verification:.8e}")
if error_verification < 1e-14:
    print("CONFIRMATION: Our exact inverse formula is correct!")
else:
    print("WARNING: The verification shows some numerical discrepancies.")

# Test with different matrix sizes
print("\n" + "=" * 60)
print("COMPARING METHODS WITH DIFFERENT MATRIX SIZES")
print("=" * 60)
print("This analysis helps us understand how each method scales with matrix size")
print("and how they compare in terms of accuracy and efficiency.")

n_values = [3, 4, 5, 6, 7]
print(f"\nTesting matrix sizes: {n_values}")
print(f"Using convergence threshold epsilon = {epsilon}")
print(f"Maximum iterations allowed: {kmax}")

results = compare_methods_with_different_n(n_values, epsilon, kmax)

# Print summary table
print("\n" + "=" * 60)
print("SUMMARY OF RESULTS")
print("=" * 60)
print("Matrix Size | Method          | Iterations | Converged? | Error vs Exact")
print("-" * 75)

for result in results:
    n = result['n']

    # Schultz method
    print(f"{n:^11} | Schultz method   | {result['iterations_schultz']:^10} | " +
          f"{'Yes' if result['converged_schultz'] else 'No':^10} | {result['error_exact_schultz']:.2e}")

    # Li and Li method 1
    print(f"{' ':^11} | Li & Li method 1 | {result['iterations_li1']:^10} | " +
          f"{'Yes' if result['converged_li1'] else 'No':^10} | {result['error_exact_li1']:.2e}")

    # Li and Li method 2
    print(f"{' ':^11} | Li & Li method 2 | {result['iterations_li2']:^10} | " +
          f"{'Yes' if result['converged_li2'] else 'No':^10} | {result['error_exact_li2']:.2e}")

    if n < len(n_values):  # Add separator between different matrix sizes
        print("-" * 75)



DERIVING AND VERIFYING THE EXACT INVERSE FORMULA
For the special matrix A, we've analyzed its pattern and determined
an exact formula for its inverse without using iterative methods.

Original matrix A:
[[1. 2. 0. 0.]
 [0. 1. 2. 0.]
 [0. 0. 1. 2.]
 [0. 0. 0. 1.]]

Exact inverse matrix derived using our formula:
[[ 1. -2.  4. -8.]
 [ 0.  1. -2.  4.]
 [ 0.  0.  1. -2.]
 [ 0.  0.  0.  1.]]

Verification: A * A^(-1) should equal the identity matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

Error in verification: 0.00000000e+00
CONFIRMATION: Our exact inverse formula is correct!

COMPARING METHODS WITH DIFFERENT MATRIX SIZES
This analysis helps us understand how each method scales with matrix size
and how they compare in terms of accuracy and efficiency.

Testing matrix sizes: [3, 4, 5, 6, 7]
Using convergence threshold epsilon = 1e-08
Maximum iterations allowed: 100

------------------------------
ANALYZING MATRIX SIZE n = 3
------------------------------
Created 3x3 

In [1]:
def pseudo_inverse_schultz(A, epsilon, kmax):
    """
    Adapt the Schultz method to calculate the Moore-Penrose pseudoinverse
    of a non-square matrix A of size m×n.

    For non-square matrices, the pseudoinverse A⁺ satisfies:
    - If m > n (tall matrix): A⁺ = (A^T·A)^(-1)·A^T
    - If m < n (wide matrix): A⁺ = A^T·(A·A^T)^(-1)

    We'll use the Schultz iterative method to compute the required inverse.
    """
    m, n = A.shape
    print(f"Input matrix dimensions: {m}×{n} (non-square)")

    if m == n:
        print("WARNING: This is actually a square matrix. Using regular inverse method.")
        V0 = initial_matrix_V0(A)
        return schultz_method(A, V0, epsilon, kmax)[0]

    # CASE 1: m > n (tall matrix, more rows than columns)
    if m > n:
        print(f"Case: Tall matrix (m={m} > n={n})")
        print("Computing pseudoinverse using A⁺ = (A^T·A)^(-1)·A^T")

        # Step 1: Compute A^T·A (which is n×n)
        ATA = A.T @ A
        print(f"Computed A^T·A with dimensions {ATA.shape}")

        # Step 2: Find the initial approximation for (A^T·A)^(-1)
        V0 = initial_matrix_V0(ATA)
        print("Calculated initial approximation V0")

        # Step 3: Use Schultz method to approximate (A^T·A)^(-1)
        V_ATA_inv, iterations, converged = schultz_method(ATA, V0, epsilon, kmax)

        if converged:
            print(f"Schultz method converged after {iterations} iterations")
        else:
            print(f"WARNING: Schultz method did not converge after {iterations} iterations")

        # Step 4: Compute A⁺ = (A^T·A)^(-1)·A^T
        A_pseudo = V_ATA_inv @ A.T
        print(f"Computed pseudoinverse with dimensions {A_pseudo.shape}")

        return A_pseudo

    # CASE 2: m < n (wide matrix, more columns than rows)
    else:
        print(f"Case: Wide matrix (m={m} < n={n})")
        print("Computing pseudoinverse using A⁺ = A^T·(A·A^T)^(-1)")

        # Step 1: Compute A·A^T (which is m×m)
        AAT = A @ A.T
        print(f"Computed A·A^T with dimensions {AAT.shape}")

        # Step 2: Find the initial approximation for (A·A^T)^(-1)
        V0 = initial_matrix_V0(AAT)
        print("Calculated initial approximation V0")

        # Step 3: Use Schultz method to approximate (A·A^T)^(-1)
        V_AAT_inv, iterations, converged = schultz_method(AAT, V0, epsilon, kmax)

        if converged:
            print(f"Schultz method converged after {iterations} iterations")
        else:
            print(f"WARNING: Schultz method did not converge after {iterations} iterations")

        # Step 4: Compute A⁺ = A^T·(A·A^T)^(-1)
        A_pseudo = A.T @ V_AAT_inv
        print(f"Computed pseudoinverse with dimensions {A_pseudo.shape}")

        return A_pseudo

# Let's test the pseudoinverse function and compare with NumPy
print("\n" + "=" * 60)
print("BONUS SECTION (15 POINTS): PSEUDOINVERSE FOR NON-SQUARE MATRICES")
print("=" * 60)
print("Adapting the Schultz method to compute the Moore-Penrose pseudoinverse")
print("for non-square matrices (m×n) and comparing with NumPy's result.")

# Example 1: Tall matrix (more rows than columns)
print("\n" + "-" * 60)
print("EXAMPLE 1: TALL MATRIX (m > n)")
print("-" * 60)
m, n = 5, 3  # 5 rows, 3 columns
A_tall = np.zeros((m, n))

# Fill with a specific pattern similar to our special matrix
for i in range(m):
    for j in range(n):
        if i == j:
            A_tall[i, j] = 1
        elif j == i + 1:
            A_tall[i, j] = 2

print("Tall matrix A (5×3):")
print(A_tall)

# Calculate pseudoinverse using our method
epsilon = 1e-8
kmax = 100
our_A_tall_pseudo = pseudo_inverse_schultz(A_tall, epsilon, kmax)

print("\nPseudoinverse A⁺ (using our Schultz method adaptation):")
print(our_A_tall_pseudo)

# Calculate pseudoinverse using NumPy for comparison
numpy_A_tall_pseudo = np.linalg.pinv(A_tall)
print("\nPseudoinverse A⁺ (using NumPy's pinv function):")
print(numpy_A_tall_pseudo)

# Compare the results
error_tall = np.linalg.norm(our_A_tall_pseudo - numpy_A_tall_pseudo, np.inf)
print(f"\nDifference between our result and NumPy's result (infinity norm): {error_tall:.2e}")
if error_tall < epsilon:
    print("EXCELLENT! Our implementation matches NumPy's result within the error tolerance.")
else:
    print("There are some differences between our implementation and NumPy's result.")
    print("This could be due to different algorithms or numerical precision.")

# Example 2: Wide matrix (more columns than rows)
print("\n" + "-" * 60)
print("EXAMPLE 2: WIDE MATRIX (m < n)")
print("-" * 60)
m, n = 3, 5  # 3 rows, 5 columns
A_wide = np.zeros((m, n))

# Fill with a specific pattern similar to our special matrix
for i in range(m):
    for j in range(n):
        if i == j:
            A_wide[i, j] = 1
        elif j == i + 1:
            A_wide[i, j] = 2

print("Wide matrix A (3×5):")
print(A_wide)

# Calculate pseudoinverse using our method
our_A_wide_pseudo = pseudo_inverse_schultz(A_wide, epsilon, kmax)

print("\nPseudoinverse A⁺ (using our Schultz method adaptation):")
print(our_A_wide_pseudo)

# Calculate pseudoinverse using NumPy for comparison
numpy_A_wide_pseudo = np.linalg.pinv(A_wide)
print("\nPseudoinverse A⁺ (using NumPy's pinv function):")
print(numpy_A_wide_pseudo)

# Compare the results
error_wide = np.linalg.norm(our_A_wide_pseudo - numpy_A_wide_pseudo, np.inf)
print(f"\nDifference between our result and NumPy's result (infinity norm): {error_wide:.2e}")
if error_wide < epsilon:
    print("EXCELLENT! Our implementation matches NumPy's result within the error tolerance.")
else:
    print("There are some differences between our implementation and NumPy's result.")
    print("This could be due to different algorithms or numerical precision.")


Function Approximation Program
----------------------------------------
Error: File 'input.txt' not found.
Creating a sample input file 'input.txt'...
Sample input file created. Edit 'input.txt' with your data and run the program again.
